# Problem 1

In [23]:
import numpy as np
import scipy.optimize
from scipy.linalg import lstsq

In [24]:
class GBM:

    def __init__(self,S0,r,sigma):
        self.S0 = 1
        self.r = 0.03
        self.sigma = 0.20

In [25]:
hw7dynamics = GBM(S0=1,r=0.03,sigma=0.20)

In [26]:
class Put:

    def __init__(self,K,T):
        self.K = K
        self.T = T

In [27]:
hw7contract = Put(K=1.1,T=4)

In [28]:
class MCengine:

    def __init__(self,M,N,seed,algorithm):

        self.M = M 
        self.N = N   
        self.rng = np.random.default_rng(seed=seed) 
        self.algorithm = algorithm


    def price_americanPut_GBM(self,contract,dynamics):

        r=dynamics.r
        sigma=dynamics.sigma
        S0=dynamics.S0

        K=contract.K
        T=contract.T

        N=self.N
        M=self.M
        dt=T/N

        Z = self.rng.normal(size=(M,N))

        paths = S0*np.exp((r-sigma**2/2)*dt*np.tile(np.arange(1,N+1),(M,1))+sigma*np.sqrt(dt)*np.cumsum(Z,axis=1))

        payoffDiscounted = np.maximum(0,K-paths[:,-1])

        for nn in np.arange(N-1,0,-1):


            X=paths[:,nn-1]
            exerciseValue = K-X

            if self.algorithm == 'value':


                basisfunctions = np.stack([np.ones(M), X, X**2], axis=1) 

                coefficients, res, rnk, s = np.linalg.lstsq(basisfunctions, payoffDiscounted, rcond=None)


                estimatedContinuationValue = basisfunctions @ coefficients  

                whichPathsToExercise = (exerciseValue >= np.maximum(estimatedContinuationValue,0))
   

            elif self.algorithm == 'policy':


                (a_opt,b_opt) = scipy.optimize.minimize(
                    negofMCaverageOfExpectedPayouts,(0,0),args=(X,exerciseValue,continuationPayoffDiscounted),method='Nelder-Mead').x

                softExerciseValues = softExercise(X, a_opt, b_opt)
                whichPathsToExercise = (exerciseValue > softExerciseValues) 

            else:
                raise ValueError('Unknown algorithm type')


            payoffDiscounted[whichPathsToExercise] = exerciseValue[whichPathsToExercise]
            payoffDiscounted[np.logical_not(whichPathsToExercise)] = continuationPayoffDiscounted[~whichPathsToExercise]
            
        continuationPayoffDiscounted = np.exp(-r*dt)*payoffDiscounted;
        estimatedContinuationValue = np.mean(continuationPayoffDiscounted);
        putprice = max(K-S0,estimatedContinuationValue);

        return(putprice)

In [29]:


def softExercise(X,a,b):
    return 1/(1+np.exp(-b*(X-a)))

In [33]:


def negofMCaverageOfExpectedPayouts(coefficients, x, exercisePayoff, continuationPayoff):

    p = softExercise(x,*coefficients)


    paths = p * exercisePayoff + (1-p) * continuationPayoff
    avg_path = -np.mean(paths)

    return avg_path


### a. 

In [34]:
hw7MC = MCengine(M=10000,N=4,seed=0,algorithm='value')
hw7MC.price_americanPut_GBM(hw7contract,hw7dynamics)

0.16074111276369632

### b. 

In [35]:
hw7MC = MCengine(M=10000,N=4,seed=0,algorithm='policy')
hw7MC.price_americanPut_GBM(hw7contract,hw7dynamics)

/var/folders/g_/x9l31p1j2rjfgwqskfjc6knw0000gn/T/ipykernel_19568/3835755103.py:7: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-b*(X-a)))


0.12142759510531755